# Automated ML

In [ ]:
import azureml.core
from azureml.core.environment import Environment
from azureml.core.conda_dependencies import CondaDependencies
from azureml.core.experiment import Experiment
from azureml.core.workspace import Workspace
from azureml.core.dataset import Dataset
from azureml.data.dataset_factory import TabularDatasetFactory
from azureml.core.compute import AmlCompute
from azureml.core.compute import ComputeTarget
from azureml.core.compute_target import ComputeTargetException
from azureml.core.model import Model
from sklearn.model_selection import train_test_split
from azureml.widgets import RunDetails
from azureml.train.automl import AutoMLConfig
import numpy as np
import pandas as pd


## Setup

In [2]:
ws = Workspace.from_config()
experiment_name = 'capstone'
experiment=Experiment(ws, experiment_name)

Performing interactive authentication. Please follow the instructions on the terminal.
To sign in, use a web browser to open the page https://microsoft.com/devicelogin and enter the code RTLM9Z7K9 to authenticate.
You have logged in. Now let us find all the subscriptions to which you have access...
Interactive authentication successfully completed.


In [3]:
amlcompute_cluster_name = "capstonecompute"

try:
    compute_target = ComputeTarget(workspace=ws, name=amlcompute_cluster_name)
    print('Found existing cluster')
except ComputeTargetException:
    compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_DS3_V2', max_nodes=4)
    compute_target = ComputeTarget.create(ws, amlcompute_cluster_name, compute_config)
    compute_target.wait_for_completion(show_output=True, min_node_count = 1, timeout_in_minutes = 10)


Creating
Succeeded........................................................

KeyboardInterrupt: 

## Dataset

### Overview
TODO: In this markdown cell, give an overview of the dataset you are using. Also mention the task you will be performing.


In [6]:
# test to see if dataset is in store
key = 'heartfailuredataset'
if key in ws.datasets.keys(): 
    dataset = ws.datasets[key] 
    print('Found dataset')

# if not, load the dataset, save it to the store
else:
    url = 'https://archive.ics.uci.edu/ml/machine-learning-databases/00519/heart_failure_clinical_records_dataset.csv'
    dataset = Dataset.Tabular.from_delimited_files(url)
    dataset = dataset.register(workspace=ws, name=key)
    
               
# split into data and test sets
ds = dataset.to_pandas_dataframe()
# use train test split
train, test = train_test_split(ds, random_state=625, shuffle=True)

# then write the train data to a file and load to datastore
train.to_csv('train.csv', index=False)
datastore = ws.get_default_datastore()
datastore.upload_files(files = ['./train.csv'])

# then read to tabular dataset for automl
train = Dataset.Tabular.from_delimited_files(path = [(datastore,'train.csv')])


Found dataset
Uploading an estimated of 1 files
Target already exists. Skipping upload for train.csv
Uploaded 0 files


## AutoML Configuration

TODO: Explain why you chose the automl settings and cofiguration you used below.

In [7]:
automl_settings = {
    "experiment_timeout_minutes":30,
    "enable_early_stopping":True,    
    "primary_metric":'accuracy',
    "n_cross_validations":5,    
}

automl_config = AutoMLConfig(
    compute_target=compute_target, 
    task='classification', 
    training_data=train,
    label_column_name='DEATH_EVENT',
    **automl_settings)

In [8]:
run = experiment.submit(automl_config)

Running on remote.


## Run Details

OPTIONAL: Write about the different models trained and their performance. Why do you think some models did better than others?


In [9]:
RunDetails(run).show()

_AutoMLWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', 's…

In [ ]:
run.wait_for_completion()

## Best Model

In [12]:
best_run, model = run.get_output()

In [13]:
print(best_run)

Run(Experiment: capstone,
Id: AutoML_310a2d92-67fc-46f9-bf09-209b6b134de8_30,
Type: azureml.scriptrun,
Status: Completed)


In [14]:
print(model)

Pipeline(memory=None,
         steps=[('datatransformer',
                 DataTransformer(enable_dnn=None, enable_feature_sweeping=None,
                                 feature_sweeping_config=None,
                                 feature_sweeping_timeout=None,
                                 featurization_config=None, force_text_dnn=None,
                                 is_cross_validation=None,
                                 is_onnx_compatible=None, logger=None,
                                 observer=None, task=None, working_dir=None)),
                ('prefittedsoftvotingclassifier',...
                                                                                                  max_samples=None,
                                                                                                  min_impurity_decrease=0.0,
                                                                                                  min_impurity_split=None,
                            

In [15]:
best_run.get_tags()

{'_aml_system_azureml.automlComponent': 'AutoML',
 '_aml_system_ComputeTargetStatus': '{"AllocationState":"steady","PreparingNodeCount":0,"RunningNodeCount":0,"CurrentNodeCount":1}',
 'ensembled_iterations': '[3, 9, 14, 23, 16, 6, 2, 21]',
 'ensembled_algorithms': "['RandomForest', 'ExtremeRandomTrees', 'XGBoostClassifier', 'XGBoostClassifier', 'LightGBM', 'GradientBoosting', 'RandomForest', 'ExtremeRandomTrees']",
 'ensemble_weights': '[0.1, 0.1, 0.2, 0.1, 0.2, 0.1, 0.1, 0.1]',
 'best_individual_pipeline_score': '0.8705050505050504',
 'best_individual_iteration': '3',
 '_aml_system_automl_is_child_run_end_telemetry_event_logged': 'True'}

In [16]:
best_run.get_metrics(name='accuracy')

{'accuracy': 0.8882828282828281}

### Test the model

In [17]:
#save the first two rows in a variable called tosend to use to test deployment and drop the output value
tosend = test[:2].values.tolist()
tosend = [tosend[0][:-1],tosend[1][:-1]]

In [18]:
# Split into x and y tests
y_test = test['DEATH_EVENT']
x_test = test.drop(['DEATH_EVENT'],axis=1)

In [19]:
# test the model and create a confusion matrix
from sklearn.metrics import confusion_matrix
ypred = model.predict(x_test)
cmatrix = confusion_matrix(y_test, ypred)
pd.DataFrame(cmatrix)

,0,1
0,44,5
1,8,18


### Register the model

In [20]:
#registering the model
# need to add properties, do later.

model = run.register_model(model_name ='automl_model')



In [21]:
print(model)

Model(workspace=Workspace.create(name='quick-starts-ws-135113', subscription_id='a24a24d5-8d87-4c8a-99b6-91ed2d2df51f', resource_group='aml-quickstarts-135113'), name=automl_model, id=automl_model:1, version=1, tags={}, properties={})


In [23]:
# print the model path
model.get_model_path(model_name='automl_model',version=1)

ERROR:azureml.core.model:Model automl_model not found in cache at azureml-models or in current working directory /mnt/batch/tasks/shared/LS_root/mounts/clusters/final/code/Users/odl_user_135113. For more info, set logging level to DEBUG.



WebserviceException: WebserviceException:
	Message: Model automl_model not found in cache at azureml-models or in current working directory /mnt/batch/tasks/shared/LS_root/mounts/clusters/final/code/Users/odl_user_135113. For more info, set logging level to DEBUG.
	InnerException None
	ErrorResponse 
{
    "error": {
        "message": "Model automl_model not found in cache at azureml-models or in current working directory /mnt/batch/tasks/shared/LS_root/mounts/clusters/final/code/Users/odl_user_135113. For more info, set logging level to DEBUG."
    }
}

## Model Deployment

Remember you have to deploy only one of the two models you trained.. Perform the steps in the rest of this notebook only if you wish to deploy this model.

TODO: In the cell below, register the model, create an inference config and deploy the model as a web service.

### Save the environment

In [24]:
# get the list of environments
Environment.list(workspace=ws).keys()


dict_keys(['AzureML-VowpalWabbit-8.8.0', 'AzureML-AutoML', 'AzureML-Hyperdrive-ForecastDNN', 'AzureML-PyTorch-1.4-CPU', 'AzureML-PyTorch-1.4-GPU', 'AzureML-AutoML-DNN', 'AzureML-AutoML-DNN-GPU', 'AzureML-Designer-Transform', 'AzureML-Designer', 'AzureML-AutoML-GPU', 'AzureML-Designer-IO', 'AzureML-Dask-CPU', 'AzureML-Designer-R', 'AzureML-Dask-GPU', 'AzureML-Designer-Recommender', 'AzureML-PyTorch-1.5-CPU', 'AzureML-PyTorch-1.5-GPU', 'AzureML-Sidecar', 'AzureML-Designer-CV-Transform', 'AzureML-Designer-Score', 'AzureML-Designer-PyTorch', 'AzureML-Designer-CV', 'AzureML-TensorFlow-2.1-GPU', 'AzureML-TensorFlow-2.1-CPU', 'AzureML-Designer-PyTorch-Train', 'AzureML-AutoML-DNN-Vision-GPU', 'AzureML-Designer-VowpalWabbit', 'AzureML-TensorFlow-2.2-GPU', 'AzureML-TensorFlow-2.2-CPU', 'AzureML-PyTorch-1.6-CPU', 'AzureML-PyTorch-1.6-GPU', 'AzureML-Triton', 'AzureML-TensorFlow-2.3-CPU', 'AzureML-TensorFlow-2.3-GPU', 'AzureML-DeepSpeed-0.3-GPU', 'AzureML-TensorFlow-2.0-CPU', 'AzureML-Chainer-5.1.0

In [37]:
from azureml.automl.core.shared import constants

best_run.download_file(constants.CONDA_ENV_FILE_PATH, 'myenv.yml')

In [26]:
#remember to insert environment name
my_env = Environment.get(workspace=ws, name="AzureML-AutoML")

#conda_dependencies = CondaDependencies()
#conda_dependencies.add_pip_package("azureml-defaults")
#conda_dependencies.add_conda_package("py-xgboost")

# writing the environment file
with open('automl_env.yml',"w") as f:
    f.write(my_env.serialize_to_string())
    


AttributeError: 'Environment' object has no attribute 'serialize_to_string'

In [38]:
os.listdir(os.curdir), os.getcwd()

(['.azureml',
  '.config',
  '.ipynb_aml_checkpoints',
  'automl.ipynb',
  'automl.log',
  'automl_env.yml',
  'azureml_automl.log',
  'heart_failure_clinical_records_dataset.csv',
  'hyperparameter_tuning.ipynb',
  'model.pkl',
  'myenv.yml',
  'README.md',
  'score.py',
  'scoring.py',
  'sla_weekday_preprocessed_azure.csv',
  'someone.ipynb',
  'someone.ipynb.amltemp',
  'train.csv'],
 '/mnt/batch/tasks/shared/LS_root/mounts/clusters/final/code/Users/odl_user_135113')

### Create a scoring script

In [39]:
%%writefile score.py

import os
import numpy as np
import json
import joblib

def init():
    global model
    model_path = ('./model.pkl')
    model = joblib.load(model_path)

def run(data):
    try:
        data = np.array(json.loads(data))
        result = model.predict(data)
        return result.tolist()
    except Exception as err:
        return str(err)

Overwriting score.py


In [40]:
# create inference_config
from azureml.core.model import InferenceConfig
inference_config = InferenceConfig(entry_script="score.py", environment = my_env)

### Deploy locally and test for testing purposes

In [29]:
from azureml.core.webservice import LocalWebService
deployment_config = LocalWebService.deploy_configuration(port=9000)

# can I use Model or do I have to use model
service = Model.deploy(workspace=ws, name="test-service",
                       models=[model],
                       inference_config=inference_config, deployment_config=deployment_config)

ImportError: cannot import name 'LocalWebService'

In [30]:
data = json.dumps({"data":tosend})
print(service.run(input_data=data))

NameError: name 'json' is not defined

In [31]:
service.delete()

NameError: name 'service' is not defined

### Deploy as a webservice

In [41]:
#set deployment_config for webservice deployment
from azureml.core.webservice import Webservice, AciWebservice
deployment_config = AciWebservice.deploy_configuration(cpu_cores=1, memory_gb = 1)

In [46]:
from azureml.core.model import InferenceConfig
from azureml.core.webservice import AciWebservice


service_name = 'my-custom-env-service2'

inference_config = InferenceConfig(entry_script='score.py', environment=best_run.get_environment())
aci_config = AciWebservice.deploy_configuration(cpu_cores=1, memory_gb=1)

service = Model.deploy(workspace=ws,
                       name=service_name,
                       models=[model],
                       inference_config=inference_config,
                       deployment_config=aci_config,
                       overwrite=True)
service.wait_for_deployment(show_output=True)

Tips: You can try get_logs(): https://aka.ms/debugimage#dockerlog or local deployment: https://aka.ms/debugimage#debug-locally to debug if deployment takes longer than 10 minutes.
Running................
Succeeded
ACI service creation operation finished, operation "Succeeded"


In [47]:
print(service.get_logs())

2021-01-17T20:41:23,274415900+00:00 - gunicorn/run 
2021-01-17T20:41:23,283700400+00:00 - rsyslog/run 
2021-01-17T20:41:23,318317900+00:00 - iot-server/run 
2021-01-17T20:41:23,355320900+00:00 - nginx/run 
/usr/sbin/nginx: /azureml-envs/azureml_8eff28b157f42edcd2424a5aae6c8074/lib/libcrypto.so.1.0.0: no version information available (required by /usr/sbin/nginx)
/usr/sbin/nginx: /azureml-envs/azureml_8eff28b157f42edcd2424a5aae6c8074/lib/libcrypto.so.1.0.0: no version information available (required by /usr/sbin/nginx)
/usr/sbin/nginx: /azureml-envs/azureml_8eff28b157f42edcd2424a5aae6c8074/lib/libssl.so.1.0.0: no version information available (required by /usr/sbin/nginx)
/usr/sbin/nginx: /azureml-envs/azureml_8eff28b157f42edcd2424a5aae6c8074/lib/libssl.so.1.0.0: no version information available (required by /usr/sbin/nginx)
/usr/sbin/nginx: /azureml-envs/azureml_8eff28b157f42edcd2424a5aae6c8074/lib/libssl.so.1.0.0: no version information available (required by /usr/sbin/nginx)
rsyslogd

In [48]:
key, _ = service.get_keys()

ERROR:azureml.core.webservice.webservice:Received bad response from Model Management Service:
Response Code: 400
Headers: {'Date': 'Sun, 17 Jan 2021 20:42:03 GMT', 'Content-Type': 'application/json', 'Transfer-Encoding': 'chunked', 'Connection': 'keep-alive', 'x-ms-client-request-id': 'b68424f2f56e4b1f8135ba513ab88570', 'x-ms-client-session-id': '84188a7c-f1a1-47ee-ac02-2ae4c9c14425', 'api-supported-versions': '1.0, 2018-03-01-preview, 2018-11-19', 'X-Content-Type-Options': 'nosniff', 'x-request-time': '0.091', 'Strict-Transport-Security': 'max-age=15724800; includeSubDomains; preload'}
Content: b'{"code":"BadRequest","statusCode":400,"message":"The request is invalid.","details":[{"code":"AuthDisabled","message":"Authentication is disabled (authEnabled set to false). Enable service authentication to list/regenerate keys. Subscription: a24a24d5-8d87-4c8a-99b6-91ed2d2df51f, ResourceGroup: aml-quickstarts-135113, Workspace: quick-starts-ws-135113, ACR: /subscriptions/a24a24d5-8d87-4c8a-9

WebserviceException: WebserviceException:
	Message: Received bad response from Model Management Service:
Response Code: 400
Headers: {'Date': 'Sun, 17 Jan 2021 20:42:03 GMT', 'Content-Type': 'application/json', 'Transfer-Encoding': 'chunked', 'Connection': 'keep-alive', 'x-ms-client-request-id': 'b68424f2f56e4b1f8135ba513ab88570', 'x-ms-client-session-id': '84188a7c-f1a1-47ee-ac02-2ae4c9c14425', 'api-supported-versions': '1.0, 2018-03-01-preview, 2018-11-19', 'X-Content-Type-Options': 'nosniff', 'x-request-time': '0.091', 'Strict-Transport-Security': 'max-age=15724800; includeSubDomains; preload'}
Content: b'{"code":"BadRequest","statusCode":400,"message":"The request is invalid.","details":[{"code":"AuthDisabled","message":"Authentication is disabled (authEnabled set to false). Enable service authentication to list/regenerate keys. Subscription: a24a24d5-8d87-4c8a-99b6-91ed2d2df51f, ResourceGroup: aml-quickstarts-135113, Workspace: quick-starts-ws-135113, ACR: /subscriptions/a24a24d5-

TODO: In the cell below, send a request to the web service you deployed to test it.

In [49]:
url = service.scoring_uri

In [51]:
import requests
import json

data = json.dumps({"data":tosend})
headers = {'Content-Type':'application/json'}
headers['Authorization'] = f'Bearer {key}'
response = requests.post(url,data,headers=headers)
print(response.text)


"Must pass 2-d input"


### Remove the webservice, models, and shut down the compute cluster

In [52]:
service.delete()
model.delete()
run.delete()
experiment.delete()

ERROR:azureml._restclient.models_client:Received bad response from Model Management Service:
Response Code: 412
Correlation: {'operation': '5b63fed83d543d4ca46dbd0150d5ed05', 'request': '584b5f4b3d01254a', 'RequestId': '584b5f4b3d01254a'}
Content: [<azureml._restclient.models.error_details.ErrorDetails object at 0x7fc22067e438>]



WebserviceException: WebserviceException:
	Message: Received bad response from Model Management Service:
Response Code: 412
Correlation: {'operation': '5b63fed83d543d4ca46dbd0150d5ed05', 'request': '584b5f4b3d01254a', 'RequestId': '584b5f4b3d01254a'}
Content: [<azureml._restclient.models.error_details.ErrorDetails object at 0x7fc22067e438>]
	InnerException None
	ErrorResponse 
{
    "error": {
        "message": "Received bad response from Model Management Service:\nResponse Code: 412\nCorrelation: {'operation': '5b63fed83d543d4ca46dbd0150d5ed05', 'request': '584b5f4b3d01254a', 'RequestId': '584b5f4b3d01254a'}\nContent: [<azureml._restclient.models.error_details.ErrorDetails object at 0x7fc22067e438>]"
    }
}

In [ ]:
compute_target.delete()
